In [65]:

#Required Libraries
import cv2
import os
import pygame.mixer as mixer
from deepface import DeepFace 
from tkinter import *
from PIL import Image,ImageTk

#Required Constants
EMOTION_LIST=['angry','sad','happy','fear']
AUDIO_PATH='audio'
BG_COLOR='#293241'
BG1_COLOR='#3d5a80'
FG_COLOR='#98c1d9'

#Initializing pygame mixer
mixer.init()

# If any error with pyimage or file location occurs (restart kernel)


In [66]:
#All Basic Button Functions

#Back Button
def Backfunction(playlist:Listbox,emotion,status_label:Label):
    #Default selection
    if not playlist.curselection():
        playlist.activate(0)
        playlist.selection_set(0)
    num=playlist.curselection()

    #Set the next index
    next_ind=(num[0]-1)%playlist.size()

    #Set location of the file
    song=f'{os.getcwd()}/{AUDIO_PATH}/{emotion}/{playlist.get(next_ind)}'

    #Load and play
    mixer.music.load(song)
    mixer.music.play()

    #Change Status
    status_label.configure(text='Now Playing...')

    #Clear and Reset selection
    playlist.selection_clear(0,END)
    playlist.activate(next_ind)
    playlist.select_set(next_ind)
    return

#Fwd Button
def Fwdfunction(playlist: Listbox,emotion,status_label:Label):
    #Default selection
    if not playlist.curselection():
        playlist.activate(0)
        playlist.selection_set(0)
    num=playlist.curselection()

    #Set the value of next index
    next_ind=(num[0]+1)%playlist.size()

    #Set location of song files
    song=f'{os.getcwd()}/{AUDIO_PATH}/{emotion}/{playlist.get(next_ind)}'

    #Load and Play
    mixer.music.load(song)
    mixer.music.play()

    #Change Status
    status_label.configure(text='Now Playing...')



    #Reset and Clear selection
    playlist.selection_clear(0,END)
    playlist.activate(next_ind)
    playlist.select_set(next_ind)
    return


#Play Button
def Playfunction(playlist:Listbox,status_label:Label,emotion,paused): 
    #Set status of the song 
    status_label.configure(text='Now Playing...')

    #Pause/Resume functionality
    if paused:
        mixer.music.unpause()
        paused=False
        return
       
    #Get in the directory
    os.chdir(f'{AUDIO_PATH}/{emotion}')
    song=playlist.get(ACTIVE)

    #Load and Play song
    mixer.music.load(song)
    mixer.music.play()

    #Get back to main directory
    os.chdir('../../')


#Pause Button
def Pausefunction(status_label:Label):
    #Pause music
    mixer.music.pause()

    #Set label to pause
    status_label.configure(text='Song Paused')
    return

#Stop button
def Stopfunction(status_label:Label):
    #Stop music
    mixer.music.stop()

    #Set label to stop
    status_label.configure(text='Music Stopped !')
    return

#Load songs by default
def load_songs(playlist:Listbox,emotion):
    #Set folder location
    os.chdir(f'{AUDIO_PATH}/{emotion}')
    tracks=os.listdir()

    #Add songs to listbox
    for track in tracks:
        playlist.insert(END,track)

    #Load and Play music
    mixer.music.load(tracks[0])
    mixer.music.play()

    #Get back to main directory
    os.chdir('../../')

    

In [67]:
def ReplaceTop():
    # Empty list for appending each toplevel
    tops = [] 

    # Looping through widgets in main window to find topelevel windows
    for widget in root.winfo_children(): 
        if '!toplevel' in str(widget):
            tops.append(widget) 

    #If more than one top level exists destroy the previous one
    if(len(tops)==1):
        tops[0].destroy()

In [68]:
#Pop-up menu(Music Player)
def MusicPlayer(emotion):
    #Set Default emotion
    if emotion not in EMOTION_LIST:
        emotion='peaceful'

    #Works with top.update() function(Set images to global)
    global fwd_img,back_img,play_img,pause_img,stop_img

    #pause variable
    paused=False

    #Avoid multiple top instances
    ReplaceTop()

    #Top Level popup
    top=Toplevel(root) 

    #Disable maximize
    top.resizable(0,0)

    #Title and size
    top.title("Mp3 Player")
    top.geometry("500x470+800+200")
    top.configure(bg=BG1_COLOR)

    #Mood status label 
    Label(top,text=f'Your current mood is : {emotion}' if emotion!='peaceful' else 'neutral/peaceful',font=("times new roman",15,"bold"),bg=BG_COLOR,fg=FG_COLOR).pack(fill=X)


    #Songs list based on emotion
    playlist=Listbox(top,bg='black',fg=FG_COLOR,width=60,font=("times new roman",14,"bold"),selectforeground='black',selectbackground=FG_COLOR)
    playlist.pack(pady=20)

    #Current Status of song label
    status_label=Label(top,text=f'Now Playing...',font=("times new roman",15,"bold"),bg=BG_COLOR,fg=FG_COLOR)
    status_label.pack(fill=X,pady=20)


    #Control Button Images
    back_img=ImageTk.PhotoImage(file='BtnImages/back50.png')
    fwd_img=PhotoImage(file='BtnImages/forward50.png')
    play_img=PhotoImage(file='BtnImages/play50.png')
    stop_img=PhotoImage(file='BtnImages/stop50.png')
    pause_img=PhotoImage(file='BtnImages/pause50.png')

    #Frame for Buttons
    control_frame=Frame(top,bg=BG1_COLOR)
    control_frame.pack(pady=20)

    #Control Buttons
    back_btn=Button(control_frame,image=back_img,borderwidth=0,command=lambda:Backfunction(playlist,emotion,status_label))
    fwd_btn=Button(control_frame,image=fwd_img,borderwidth=0,command=lambda:Fwdfunction(playlist,emotion,status_label))
    play_btn=Button(control_frame,image=play_img,borderwidth=0,command=lambda: Playfunction(playlist,status_label,emotion,paused))
    pause_btn=Button(control_frame,image=pause_img,borderwidth=0,command=lambda : Pausefunction(status_label))
    stop_btn=Button(control_frame,image=stop_img,borderwidth=0,command=lambda:Stopfunction(status_label))

    #Control Button positions
    back_btn.grid(row=0,column=0,padx=10)        
    fwd_btn.grid(row=0,column=1,padx=10)
    play_btn.grid(row=0,column=2,padx=10)
    pause_btn.grid(row=0,column=3,padx=10)
    stop_btn.grid(row=0,column=4,padx=10)

    #load all songs 
    load_songs(playlist,emotion)
    
    top.update()

In [69]:
#On closing the main window
def DestroyCamera():
    #release cv2 camera and destroy all windows
    cam.release()
    cv2.destroyAllWindows()

    #Stop pygame mixer music
    mixer.music.fadeout(0)

    #Destroy tkinter window
    root.destroy()


In [70]:
#To Recognize emotion from the image
def RecognizeEmotion():
    #Get the image
    image=Image.fromarray(img_rgb)

    #Save the image
    image.save("mood.jpg")

    #Analyze the image
    result=DeepFace.analyze("mood.jpg",actions=['emotion'],enforce_detection=False)

    #Return the result
    MusicPlayer(result['dominant_emotion'])

In [71]:
#Basic Setup
root=Tk()
root.title('Mood Synergizer')
root.geometry("700x740+100+0")
root.resizable(0,0)
root.configure(bg=BG1_COLOR)

#Path switcher (to get back to current directory after first play)

#Basic Labels
Label(root,text="Mood Synergizer",font=("times new roman",30,"bold"),bg=BG_COLOR,fg=FG_COLOR).pack(fill=X)
Label(root,text="(Curates a playlist according to your mood!, Just take a picture)",font=("times new roman",15),bg=BG_COLOR,fg=FG_COLOR).pack(fill=X)

#Frame and labels 
main_frame=LabelFrame(root,bg=BG1_COLOR)
main_frame.pack(pady=40)
label_1=Label(main_frame,bg=BG_COLOR)
label_1.pack()

#Capture video
cam=cv2.VideoCapture(0)

#Capture button
Button(text="Take Snapshot",font=("times new roman",20,"bold"),bg=BG_COLOR,fg=FG_COLOR,command=RecognizeEmotion).pack(fill=X,expand=True,pady=20)

while True:
    #Get the image from array
    img=cam.read()[1]

    #get RGB color convention
    img_rgb=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

    #Make it compatible with Tkinter
    img=ImageTk.PhotoImage(Image.fromarray(img_rgb))

    #Insert the image
    label_1['image']=img

    root.protocol('WM_DELETE_WINDOW',DestroyCamera)

    #Update when necessary
    root.update()

error: OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
